# DuckDuckGoSearch

This notebook provides a quick overview for getting started with [DuckDuckGoSearch](/docs/integrations/tools/). For detailed documentation of all DuckDuckGoSearch features and configurations head to the [API reference](https://api.js.langchain.com/classes/langchain_community_tools_duckduckgo_search.DuckDuckGoSearch.html).

DuckDuckGoSearch offers a privacy-focused search API designed for LLM Agents. It provides seamless integration with a wide range of data sources, prioritizing user privacy and relevant search results.

## Overview

### Integration details

| Class | Package | [PY support](https://python.langchain.com/docs/integrations/tools/ddg/) | Package latest |
| :--- | :--- | :---: | :---: |
| [DuckDuckGoSearch](https://api.js.langchain.com/classes/langchain_community_tools_duckduckgo_search.DuckDuckGoSearch.html) | [`@langchain/community`](https://www.npmjs.com/package/@langchain/community) | ✅ |  ![NPM - Version](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |

## Setup

The integration lives in the `@langchain/community` package, along with the `duck-duck-scrape` dependency:

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community @langchain/core duck-duck-scrape
</Npm2Yarn>
```

### Credentials

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability:

```typescript
process.env.LANGCHAIN_TRACING_V2="true"
process.env.LANGCHAIN_API_KEY="your-api-key"
```

## Instantiation

You can instantiate an instance of the `DuckDuckGoSearch` tool like this:

In [1]:
import { DuckDuckGoSearch } from "@langchain/community/tools/duckduckgo_search"

const tool = new DuckDuckGoSearch({ maxResults: 1 })

## Invocation

### [Invoke directly with args](/docs/concepts/tools)

In [2]:
await tool.invoke("what is the current weather in sf?")

[{"title":"San Francisco, CA Current Weather | AccuWeather","link":"https://www.accuweather.com/en/us/san-francisco/94103/current-weather/347629","snippet":"<b>Current</b> <b>weather</b> <b>in</b> San Francisco, CA. Check <b>current</b> conditions in San Francisco, CA with radar, hourly, and more."}]


### [Invoke with ToolCall](/docs/concepts/tools)

We can also invoke the tool with a model-generated `ToolCall`, in which case a `ToolMessage` will be returned:

In [3]:
// This is usually generated by a model, but we'll create a tool call directly for demo purposes.
const modelGeneratedToolCall = {
  args: {
    input: "what is the current weather in sf?"
  },
  id: "tool_call_id",
  name: tool.name,
  type: "tool_call",
}
await tool.invoke(modelGeneratedToolCall)

ToolMessage {
  "content": "[{\"title\":\"San Francisco, CA Weather Conditions | Weather Underground\",\"link\":\"https://www.wunderground.com/weather/us/ca/san-francisco\",\"snippet\":\"San Francisco <b>Weather</b> Forecasts. <b>Weather</b> Underground provides local & long-range <b>weather</b> forecasts, weatherreports, maps & tropical <b>weather</b> conditions for the San Francisco area.\"}]",
  "name": "duckduckgo-search",
  "additional_kwargs": {},
  "response_metadata": {},
  "tool_call_id": "tool_call_id"
}


## Chaining

We can use our tool in a chain by first binding it to a [tool-calling model](/docs/how_to/tool_calling/) and then calling it:

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```


In [1]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai"

const llm = new ChatOpenAI({
  model: "gpt-4o-mini",
})

In [10]:
import { HumanMessage } from "@langchain/core/messages";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnableLambda } from "@langchain/core/runnables";

const prompt = ChatPromptTemplate.fromMessages(
  [
    ["system", "You are a helpful assistant."],
    ["placeholder", "{messages}"],
  ]
)

const llmWithTools = llm.bindTools([tool]);

const chain = prompt.pipe(llmWithTools);

const toolChain = RunnableLambda.from(
  async (userInput: string, config) => {
    const humanMessage = new HumanMessage(userInput,);
    const aiMsg = await chain.invoke({
      messages: [new HumanMessage(userInput)],
    }, config);
    const toolMsgs = await tool.batch(aiMsg.tool_calls, config);
    return chain.invoke({
      messages: [humanMessage, aiMsg, ...toolMsgs],
    }, config);
  }
);

const toolChainResult = await toolChain.invoke("how many people have climbed mount everest?");

In [11]:
const { tool_calls, content } = toolChainResult;

console.log("AIMessage", JSON.stringify({
  tool_calls,
  content,
}, null, 2));

AIMessage {
  "tool_calls": [],
  "content": "As of December 2023, a total of 6,664 different people have reached the summit of Mount Everest."
}


## Agents

For guides on how to use LangChain tools in agents, see the [LangGraph.js](https://langchain-ai.github.io/langgraphjs/) docs.

## API reference

For detailed documentation of all DuckDuckGoSearch features and configurations head to the [API reference](https://api.js.langchain.com/classes/langchain_community_tools_duckduckgo_search.DuckDuckGoSearch.html)